In [1]:
import pandas as pd
import pingouin as pg
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, kendalltau
from read_data import *

from sdt_constrained_probit_glmm import (
    prepare_sdt_dataframe, sdt_constrained_formula,
    fit_sdt_probit_glm, reference_delta_grid,
    sdt_from_model
)

%load_ext autoreload
%autoreload 2

## Load and preprocess the data

In [2]:
allsubjdata = pd.read_csv('data/allsubjdata.csv')

In [3]:
allsubjdata.loc[allsubjdata['corr_resp']=='f','int_diff'] = 0.0

In [4]:
allsubjdata['abs_diff'] = allsubjdata['int_diff'].abs()

In [5]:
exclude_subjs = subject_exclusion(allsubjdata)
allsubjdata_noexcl = allsubjdata.copy()
allsubjdata = allsubjdata[~allsubjdata.subject.isin(exclude_subjs)]
print(f'Excluded {len(exclude_subjs)} out of {allsubjdata_noexcl.subject.nunique()} subjects.')
print('Final sample size:', allsubjdata.subject.nunique())

Excluded 126 out of 277 subjects.
Final sample size: 151


In [6]:
allsubjdata

,scene,expected,initview,finalview,img_2,img_3,img_4,probe_1,probe_2,int_diff,p_exp,response,corr_resp,hit,rt,subject,date,time,abs_diff
384,7.0,0.0,-1.0,90.0,Zoom_Sequences_FixedLight/Scene7_-1.0_0020.png,Zoom_Sequences_FixedLight/Scene7_-1.0_0040.png,Zoom_Sequences_FixedLight/Scene7_-1.0_0055.png,Translate_Views_FixedLight/Scene7_2.0_0090_0.0...,Translate_Views_FixedLight/Scene7_2.0_0090_0.5...,0.50,0.75,f,j,0.0,1530.0,914,20220511,1652269532447,0.50
385,1.0,1.0,-1.0,30.0,Zoom_Sequences_FixedLight/Scene1_-1.0_0015.png,Zoom_Sequences_FixedLight/Scene1_-1.0_0020.png,Zoom_Sequences_FixedLight/Scene1_-1.0_0025.png,Translate_Views_FixedLight/Scene1_-1.0_0030_0....,Translate_Views_FixedLight/Scene1_-1.0_0030_0....,0.55,0.75,f,j,0.0,1326.3,914,20220511,1652269532447,0.55
386,10.0,1.0,-1.0,90.0,Zoom_Sequences_FixedLight/Scene10_-1.0_0035.png,Zoom_Sequences_FixedLight/Scene10_-1.0_0050.png,Zoom_Sequences_FixedLight/Scene10_-1.0_0055.png,Translate_Views_FixedLight/Scene10_-1.0_0090_-...,Translate_Views_FixedLight/Scene10_-1.0_0090_-...,0.00,0.75,f,f,1.0,1416.0,914,20220511,1652269532447,0.00
387,3.0,1.0,2.0,90.0,Zoom_Sequences_FixedLight/Scene3_2.0_0015.png,Zoom_Sequences_FixedLight/Scene3_2.0_0020.png,Zoom_Sequences_FixedLight/Scene3_2.0_0030.png,Translate_Views_FixedLight/Scene3_2.0_0090_0.0...,Translate_Views_FixedLight/Scene3_2.0_0090_0.0...,0.00,0.75,f,f,1.0,1727.5,914,20220511,1652269532447,0.00
388,1.0,0.0,2.0,30.0,Zoom_Sequences_FixedLight/Scene1_2.0_0015.png,Zoom_Sequences_FixedLight/Scene1_2.0_0020.png,Zoom_Sequences_FixedLight/Scene1_2.0_0025.png,Translate_Views_FixedLight/Scene1_-1.0_0030_0....,Translate_Views_FixedLight/Scene1_-1.0_0030_0....,0.55,0.75,j,j,1.0,1468.4,914,20220511,1652269532447,0.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53179,1.0,0.0,-1.0,30.0,Zoom_Sequences_FixedLight/Scene1_-1.0_0015.png,Zoom_Sequences_FixedLight/Scene1_-1.0_0020.png,Zoom_Sequences_FixedLight/Scene1_-1.0_0025.png,Translate_Views_FixedLight/Scene1_2.0_0030_0.0...,Translate_Views_FixedLight/Scene1_2.0_0030_1.0...,0.95,0.25,NaN,j,NaN,NaN,1459,20220910,1662807993840,0.95
53180,7.0,0.0,-1.0,90.0,Zoom_Sequences_FixedLight/Scene7_-1.0_0025.png,Zoom_Sequences_FixedLight/Scene7_-1.0_0030.png,Zoom_Sequences_FixedLight/Scene7_-1.0_0050.png,Translate_Views_FixedLight/Scene7_2.0_0090_-0....,Translate_Views_FixedLight/Scene7_2.0_0090_-1....,-0.95,0.25,j,j,1.0,475.6,1459,20220910,1662807993840,0.95
53181,1.0,0.0,2.0,90.0,Zoom_Sequences_FixedLight/Scene1_2.0_0015.png,Zoom_Sequences_FixedLight/Scene1_2.0_0020.png,Zoom_Sequences_FixedLight/Scene1_2.0_0040.png,Translate_Views_FixedLight/Scene1_-1.0_0090_0....,Translate_Views_FixedLight/Scene1_-1.0_0090_0....,0.00,0.25,f,f,1.0,398.3,1459,20220910,1662807993840,0.00
53182,3.0,0.0,-1.0,90.0,Zoom_Sequences_FixedLight/Scene3_-1.0_0020.png,Zoom_Sequences_FixedLight/Scene3_-1.0_0045.png,Zoom_Sequences_FixedLight/Scene3_-1.0_0060.png,Translate_Views_FixedLight/Scene3_2.0_0090_0.0...,Translate_Views_FixedLight/Scene3_2.0_0090_0.0...,0.00,0.25,f,f,1.0,344.6,1459,20220910,1662807993840,0.00


In [7]:
allsubjdata = allsubjdata.dropna(subset = ['response'])

In [8]:
trial_type = ['same' if r == 'f' else 'diff' for r in allsubjdata.corr_resp.values]
allsubjdata.loc[:, 'trial_type'] = trial_type

resp_diff = [0 if r == 'f' else 1 for r in allsubjdata.response.values]
allsubjdata.loc[:, 'resp_diff'] = resp_diff

/var/folders/8d/bvdwvdxj3s98cz8_ylgl_9j00000gn/T/ipykernel_7992/1287962830.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allsubjdata.loc[:, 'trial_type'] = trial_type
/var/folders/8d/bvdwvdxj3s98cz8_ylgl_9j00000gn/T/ipykernel_7992/1287962830.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allsubjdata.loc[:, 'resp_diff'] = resp_diff


In [9]:
df_sdt = prepare_sdt_dataframe(allsubjdata)

In [10]:
df_sdt

,scene,cond,initview,finalview,img_2,img_3,img_4,probe_1,probe_2,int_diff,...,corr_resp,hit,rt,subj,date,time,delta,trial_type,resp_diff,is_diff
384,7.0,0,-1.0,90.0,Zoom_Sequences_FixedLight/Scene7_-1.0_0020.png,Zoom_Sequences_FixedLight/Scene7_-1.0_0040.png,Zoom_Sequences_FixedLight/Scene7_-1.0_0055.png,Translate_Views_FixedLight/Scene7_2.0_0090_0.0...,Translate_Views_FixedLight/Scene7_2.0_0090_0.5...,0.50,...,j,0.0,1530.0,914,20220511,1652269532447,0.50,diff,0,1
385,1.0,1,-1.0,30.0,Zoom_Sequences_FixedLight/Scene1_-1.0_0015.png,Zoom_Sequences_FixedLight/Scene1_-1.0_0020.png,Zoom_Sequences_FixedLight/Scene1_-1.0_0025.png,Translate_Views_FixedLight/Scene1_-1.0_0030_0....,Translate_Views_FixedLight/Scene1_-1.0_0030_0....,0.55,...,j,0.0,1326.3,914,20220511,1652269532447,0.55,diff,0,1
386,10.0,1,-1.0,90.0,Zoom_Sequences_FixedLight/Scene10_-1.0_0035.png,Zoom_Sequences_FixedLight/Scene10_-1.0_0050.png,Zoom_Sequences_FixedLight/Scene10_-1.0_0055.png,Translate_Views_FixedLight/Scene10_-1.0_0090_-...,Translate_Views_FixedLight/Scene10_-1.0_0090_-...,0.00,...,f,1.0,1416.0,914,20220511,1652269532447,0.00,same,0,0
387,3.0,1,2.0,90.0,Zoom_Sequences_FixedLight/Scene3_2.0_0015.png,Zoom_Sequences_FixedLight/Scene3_2.0_0020.png,Zoom_Sequences_FixedLight/Scene3_2.0_0030.png,Translate_Views_FixedLight/Scene3_2.0_0090_0.0...,Translate_Views_FixedLight/Scene3_2.0_0090_0.0...,0.00,...,f,1.0,1727.5,914,20220511,1652269532447,0.00,same,0,0
388,1.0,0,2.0,30.0,Zoom_Sequences_FixedLight/Scene1_2.0_0015.png,Zoom_Sequences_FixedLight/Scene1_2.0_0020.png,Zoom_Sequences_FixedLight/Scene1_2.0_0025.png,Translate_Views_FixedLight/Scene1_-1.0_0030_0....,Translate_Views_FixedLight/Scene1_-1.0_0030_0....,0.55,...,j,1.0,1468.4,914,20220511,1652269532447,0.55,diff,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53178,3.0,1,2.0,90.0,Zoom_Sequences_FixedLight/Scene3_2.0_0025.png,Zoom_Sequences_FixedLight/Scene3_2.0_0030.png,Zoom_Sequences_FixedLight/Scene3_2.0_0055.png,Translate_Views_FixedLight/Scene3_2.0_0090_0.0...,Translate_Views_FixedLight/Scene3_2.0_0090_-0....,-0.95,...,j,1.0,481.2,1459,20220910,1662807993840,0.95,diff,1,1
53180,7.0,0,-1.0,90.0,Zoom_Sequences_FixedLight/Scene7_-1.0_0025.png,Zoom_Sequences_FixedLight/Scene7_-1.0_0030.png,Zoom_Sequences_FixedLight/Scene7_-1.0_0050.png,Translate_Views_FixedLight/Scene7_2.0_0090_-0....,Translate_Views_FixedLight/Scene7_2.0_0090_-1....,-0.95,...,j,1.0,475.6,1459,20220910,1662807993840,0.95,diff,1,1
53181,1.0,0,2.0,90.0,Zoom_Sequences_FixedLight/Scene1_2.0_0015.png,Zoom_Sequences_FixedLight/Scene1_2.0_0020.png,Zoom_Sequences_FixedLight/Scene1_2.0_0040.png,Translate_Views_FixedLight/Scene1_-1.0_0090_0....,Translate_Views_FixedLight/Scene1_-1.0_0090_0....,0.00,...,f,1.0,398.3,1459,20220910,1662807993840,0.00,same,0,0
53182,3.0,0,-1.0,90.0,Zoom_Sequences_FixedLight/Scene3_-1.0_0020.png,Zoom_Sequences_FixedLight/Scene3_-1.0_0045.png,Zoom_Sequences_FixedLight/Scene3_-1.0_0060.png,Translate_Views_FixedLight/Scene3_2.0_0090_0.0...,Translate_Views_FixedLight/Scene3_2.0_0090_0.0...,0.00,...,f,1.0,344.6,1459,20220910,1662807993840,0.00,same,0,0


In [12]:
formula = sdt_constrained_formula()

In [13]:
glm_res = fit_sdt_probit_glm(df_sdt, formula=formula, cluster_col='subj')

/opt/anaconda3/envs/scenecontext/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The probit link alias is deprecated. Use Probit instead. The probit link alias will be removed after the 0.15.0 release.
  warnings.warn(


In [14]:
glm_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:              resp_diff   No. Observations:                28849
Model:                            GLM   Df Residuals:                    28842
Model Family:                Binomial   Df Model:                            6
Link Function:                 probit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -17801.
Date:                Sun, 09 Nov 2025   Deviance:                       35602.
Time:                        14:09:15   Pearson chi2:                 2.88e+04
No. Iterations:                     5   Pseudo R-squ. (CS):             0.1332
Covariance Type:              cluster                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     -0.7443      0.038    -19.385      0.000      -0.820      -0.669
C(cond)[T.0]                   0.2010      0.034      5.883      0.000       0.134       0.268
C(group)[T.0.5]                0.0140      0.041      0.340      0.734      -0.067       0.095
C(group)[T.0.75]               0.0473      0.043      1.111      0.266      -0.036       0.131
is_diff                        0.9128      0.063     14.538      0.000       0.790       1.036
is_diff:delta                  0.1038      0.093      1.111      0.267      -0.079       0.287
is_diff:delta:C(cond)[T.0]    -0.0636      0.047     -1.347      0.178      -0.156       0.029
==============================================================================================
"""

In [16]:
dgrid = reference_delta_grid(df_sdt, lo=0.2, hi=0.8, n=7)

In [17]:
dgrid

array([0.   , 0.   , 0.   , 0.175, 0.55 , 0.7  , 0.85 ])

In [20]:
out = sdt_from_model(glm_res, df_sdt, dgrid=dgrid, cond_levels=(1, 0))

In [21]:
out

{1: {'H': 0.587958912002186,
  'FA': 0.23457846810167818,
  'dprime': np.float64(0.9461491111718603),
  'criterion': np.float64(0.2507768986806844)},
 0: {'H': 0.6564223392911576,
  'FA': 0.3005293418957524,
  'dprime': np.float64(0.9255971791649285),
  'criterion': np.float64(0.060080087787280106)}}